In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline

In [42]:
usersDataTrain = pd.read_csv('../setEntrenamiento.csv')


In [43]:
usersDataTrain.head()

,Unnamed: 0,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,screen_resolution,Busca_terminos_populares,...,sku_6357.0,sku_6371.0,compraPopular,usuario_Checkeador,usuario_susceptible_campaign,usuario_buscador,usuario_dubitativo,usuario_poco_util,usuario_muy_activo,usuario_convertidor
0,0,0008ed71,3,3.0,0.0,0.0,0.0,1,0.0,0,...,0.0,0.0,0.0,1,0,0,False,False,False,0
1,1,000c79fe,1,1.0,0.0,1.0,0.0,1,0.0,8,...,0.0,0.0,1.0,0,0,0,False,True,False,0
2,2,001802e4,1,1.0,0.0,5.0,0.0,1,0.0,0,...,0.0,0.0,1.0,0,0,0,False,True,False,0
3,3,0019e639,21,15.0,2.0,29.0,0.0,1,0.0,3,...,0.0,0.0,0.0,1,1,1,True,False,True,1
4,4,001b0bf9,1,2.0,1.0,0.0,0.0,1,0.0,0,...,0.0,0.0,1.0,1,0,0,False,False,False,0


In [171]:
total_train= usersDataTrain.drop(['Unnamed: 0', 'person',
       'event_ad campaign hit','screen_resolution', 'Busca_terminos_populares',
       'otros', 'marca_apple', 'marca_asus',
       'marca_lg', 'marca_motorola', 'marca_samsung', 'marca_sony',
       'condition_Bom', 'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2',
       'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB', 'Parte del dia_Madrugada', 'Parte del dia_Mañana',
       'Parte del dia_Noche', 'Parte del dia_Tarde','channel_Direct',
       'channel_Paid', 'channel_Organic', 'channel_Referral', 'Brasilero',
       'color_Dourado', 'color_Branco',
        'cantidad_De_Compras_De_Mismo_Sku',
       'Cantidad_De_Busquedas_Por_Termino', 'search_and_purchase',
       'cantidad_de_Url_de_compra', 'cantidadDeStaticPage', 'Prob_Apple',
       'Prob_Asus', 'Prob_Lenovo', 'Prob_motorola', 'Prob_quantum',
       'Prob_samsung',  'sku_multiple', 'prob_marca',
       'prob_model', 'prob_marca_checkout', 'usuario_Checkeador',
       'usuario_susceptible_campaign',
       'usuario_dubitativo',
       'prob_model_en_base_checkout', 'sku_0.0', 'sku_2831.0',
        'compraPopular', 'event_lead',
        'count_activity',
        'color_Preto', 'Prob_lg',
       'Prob_sony','usuario_poco_util']
       ,axis=1)

In [172]:
total_train.columns


Index(['cant_seciones', 'event_checkout', 'event_conversion', 'skus',
       'Computer', 'Smartphone', 'marca_lenovo', 'campaign_source_google',
       'campaign_source_criteo', 'campaign_source_rtbhouse', 'ventas_region',
       'conversiones_15lastdays', 'checkout_15lastdays', 'viewed_15lastday',
       'color_Cinza espacial', 'new_15lastdays', 'sku', 'label',
       'prob_marca_en_base_checkout', 'sku_3371.0', 'sku_6357.0', 'sku_6371.0',
       'usuario_buscador', 'usuario_muy_activo', 'usuario_convertidor'],
      dtype='object')

In [173]:
users1s = total_train[total_train["label"] == 1]
users0s = total_train[total_train["label"] == 0].head(980)
total_train = pd.concat([users0s,users1s], ignore_index=True)

total_train["label"].value_counts()

1    980
0    980
Name: label, dtype: int64

In [174]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(total_train.drop(['label'], axis=1).values, total_train["label"] , test_size=0.20, random_state=75)



In [175]:

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 30)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 7)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 227, 255, 282, 310, 337, 365, 393, 420, 448, 475, 503, 531, 558, 586, 613, 641, 668, 696, 724, 751, 779, 806, 834, 862, 889, 917, 944, 972, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 26, 43, 60, 76, 93, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [176]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
# Create a random forest Classifier. By convention, clf means 'Classifier'
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
from sklearn.metrics import make_scorer, roc_auc_score
auc = make_scorer(roc_auc_score)

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = clf, param_distributions = random_grid, n_iter = 700, cv = 5, verbose=2, random_state=42, n_jobs = -1,scoring = auc)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 700 candidates, totalling 3500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 12.4min
[Parallel(n_jobs=-1)]: Done 3500 out of 3500 | elapsed: 13.5min finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid='warn', n_iter=700, n_jobs=-1,
          param_distributions={'n_estimators': [200, 227, 255, 282, 310, 337, 365, 393, 420, 448, 475, 503, 531, 558, 586, 613, 641, 668, 696, 724, 751, 779, 806, 834, 862, 889, 917, 944, 972, 1000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 26, 43, 60, 76, 93, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, re

In [177]:
rf_random.best_params_


{'n_estimators': 282,
 'min_samples_split': 10,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 43,
 'bootstrap': True}

In [178]:
clf = RandomForestClassifier(bootstrap= True,
 max_depth= 43,
 max_features =  'sqrt',
 min_samples_leaf=4,
 min_samples_split= 10,
 n_estimators= 282
  )                            

In [179]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=43, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=10,
            min_weight_fraction_leaf=0.0, n_estimators=282, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [180]:
result = clf.predict_proba(X_test)

In [181]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [182]:
from sklearn import metrics
pred = result2
fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
metrics.auc(fpr, tpr)

0.8963990939151717

In [183]:
from sklearn.datasets import make_classification
from sklearn.ensemble import ExtraTreesClassifier
importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))


Feature ranking:
1. feature 17 (0.234477)
2. feature 12 (0.183981)
3. feature 1 (0.129493)
4. feature 13 (0.091535)
5. feature 0 (0.068814)
6. feature 7 (0.042724)
7. feature 15 (0.038014)
8. feature 14 (0.036535)
9. feature 21 (0.027714)
10. feature 4 (0.019883)
11. feature 10 (0.018792)
12. feature 16 (0.018471)
13. feature 8 (0.016725)
14. feature 3 (0.013208)
15. feature 5 (0.013127)
16. feature 2 (0.013066)
17. feature 9 (0.011200)
18. feature 11 (0.007509)
19. feature 23 (0.007081)
20. feature 22 (0.006353)
21. feature 20 (0.001009)
22. feature 6 (0.000256)
23. feature 18 (0.000034)
24. feature 19 (0.000000)


In [184]:
test = pd.read_csv('../setTest.csv')

In [185]:
person = test["person"]

In [186]:
test.head()

,Unnamed: 0,person,cant_seciones,event_checkout,event_conversion,event_ad campaign hit,event_lead,skus,screen_resolution,Busca_terminos_populares,...,sku_6357.0,sku_6371.0,compraPopular,usuario_Checkeador,usuario_susceptible_campaign,usuario_buscador,usuario_dubitativo,usuario_poco_util,usuario_muy_activo,usuario_convertidor
0,0,4886f805,1,1.0,0.0,0.0,0.0,1,0.0,0,...,0.0,0.0,1.0,0,0,0,False,True,False,0
1,1,0297fc1e,96,7.0,0.0,29.0,1.0,1,0.0,5,...,0.0,0.0,1.0,1,1,1,True,False,False,0
2,2,2d681dd8,2,1.0,0.0,1.0,0.0,1,0.0,1,...,0.0,0.0,0.0,0,0,0,False,True,False,0
3,3,cccea85e,28,1.0,0.0,15.0,0.0,1,0.0,0,...,0.0,0.0,0.0,0,1,1,False,False,False,0
4,4,4c8a8b93,20,2.0,0.0,14.0,0.0,1,0.0,9,...,0.0,0.0,1.0,1,1,1,True,False,False,0


In [188]:
test = test.drop(['Unnamed: 0', 'person',
       'event_ad campaign hit','screen_resolution', 'Busca_terminos_populares',
       'otros', 'marca_apple', 'marca_asus',
       'marca_lg', 'marca_motorola', 'marca_samsung', 'marca_sony',
       'condition_Bom', 'condition_Bom - Sem Touch ID', 'condition_Excelente',
       'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5',
       'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s',
       'version_iphone 6s', 'version_moto g4', 'version_moto g2',
       'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB',
       'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB',
       'storage_8GB', 'Parte del dia_Madrugada', 'Parte del dia_Mañana',
       'Parte del dia_Noche', 'Parte del dia_Tarde','channel_Direct',
       'channel_Paid', 'channel_Organic', 'channel_Referral', 'Brasilero',
       'color_Dourado', 'color_Branco',
        'cantidad_De_Compras_De_Mismo_Sku',
       'Cantidad_De_Busquedas_Por_Termino', 'search_and_purchase',
       'cantidad_de_Url_de_compra', 'cantidadDeStaticPage', 'Prob_Apple',
       'Prob_Asus', 'Prob_Lenovo', 'Prob_motorola', 'Prob_quantum',
       'Prob_samsung',  'sku_multiple', 'prob_marca',
       'prob_model', 'prob_marca_checkout', 'usuario_Checkeador',
       'usuario_susceptible_campaign',
       'usuario_dubitativo',
       'prob_model_en_base_checkout', 'sku_0.0', 'sku_2831.0',
        'compraPopular', 'event_lead',
        'count_activity',
        'color_Preto', 'Prob_lg',
       'Prob_sony','usuario_poco_util']
       ,axis=1)

In [189]:
test.head()

,cant_seciones,event_checkout,event_conversion,skus,Computer,Smartphone,marca_lenovo,campaign_source_google,campaign_source_criteo,campaign_source_rtbhouse,...,color_Cinza espacial,new_15lastdays,sku,prob_marca_en_base_checkout,sku_3371.0,sku_6357.0,sku_6371.0,usuario_buscador,usuario_muy_activo,usuario_convertidor
0,1,1.0,0.0,1,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.213106,0.0,0.0,0.0,0,False,0
1,96,7.0,0.0,1,0.0,1.0,0.0,4.0,5.0,18.0,...,86.0,0.0,0.0,0.204938,0.0,0.0,0.0,1,False,0
2,2,1.0,0.0,1,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.064625,0.0,0.0,0.0,0,False,0
3,28,1.0,0.0,1,1.0,0.0,0.0,11.0,2.0,1.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,1,False,0
4,20,2.0,0.0,1,0.0,1.0,0.0,8.0,0.0,1.0,...,24.0,1.0,0.0,0.273250,0.0,0.0,0.0,1,False,0


In [190]:
test.columns

Index(['cant_seciones', 'event_checkout', 'event_conversion', 'skus',
       'Computer', 'Smartphone', 'marca_lenovo', 'campaign_source_google',
       'campaign_source_criteo', 'campaign_source_rtbhouse', 'ventas_region',
       'conversiones_15lastdays', 'checkout_15lastdays', 'viewed_15lastday',
       'color_Cinza espacial', 'new_15lastdays', 'sku',
       'prob_marca_en_base_checkout', 'sku_3371.0', 'sku_6357.0', 'sku_6371.0',
       'usuario_buscador', 'usuario_muy_activo', 'usuario_convertidor'],
      dtype='object')

In [191]:
test.shape

(19415, 24)

In [192]:
nombreCol = []
for namecolumn in usersDataTrain.columns:
     if ( not namecolumn in test.columns):
            nombreCol.append(namecolumn)
print(nombreCol)
for namecolumn in test.columns:
     if ( not namecolumn in usersDataTrain.columns):
            nombreCol.append(namecolumn)
print(nombreCol)

['Unnamed: 0', 'person', 'event_ad campaign hit', 'event_lead', 'screen_resolution', 'Busca_terminos_populares', 'otros', 'marca_apple', 'marca_asus', 'marca_lg', 'marca_motorola', 'marca_samsung', 'marca_sony', 'condition_Bom', 'condition_Bom - Sem Touch ID', 'condition_Excelente', 'condition_Muito Bom', 'version_galaxy j7', 'version_galaxy j5', 'version_galaxy s7', 'version_iphone 6', 'version_iphone 5s', 'version_iphone 6s', 'version_moto g4', 'version_moto g2', 'version_galaxy s6', 'storage_128GB', 'storage_16GB', 'storage_256GB', 'storage_32GB', 'storage_4GB', 'storage_512MB', 'storage_64GB', 'storage_8GB', 'Parte del dia_Madrugada', 'Parte del dia_Mañana', 'Parte del dia_Noche', 'Parte del dia_Tarde', 'channel_Direct', 'channel_Paid', 'channel_Organic', 'channel_Referral', 'Brasilero', 'count_activity', 'color_Preto', 'color_Dourado', 'color_Branco', 'cantidad_De_Compras_De_Mismo_Sku', 'Cantidad_De_Busquedas_Por_Termino', 'search_and_purchase', 'cantidad_de_Url_de_compra', 'canti

In [193]:
result = clf.predict_proba(test)

In [194]:
result2 = []
for elem in result:
    result2.append(elem[1])

In [195]:
person = person.to_frame()

In [196]:
person['label'] = result2

In [197]:
person.head()

,person,label
0,4886f805,0.029756
1,0297fc1e,0.486578
2,2d681dd8,0.139271
3,cccea85e,0.748583
4,4c8a8b93,0.585443


In [198]:
(person["label"]<0.5).value_counts()

True     14131
False     5284
Name: label, dtype: int64

In [199]:
kaggle = pd.read_csv('../data/trocafone_kaggle_test.csv')

In [200]:
kaggle.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [201]:
resultado = pd.merge(kaggle,person,on="person",how = "left")

In [202]:
resultado.head()

,person,label
0,4886f805,0.029756
1,0297fc1e,0.486578
2,2d681dd8,0.139271
3,cccea85e,0.748583
4,4c8a8b93,0.585443


In [203]:
resultado["label"].isnull().value_counts()

False    19415
Name: label, dtype: int64

In [204]:
resultado.to_csv('resultadoRandomForestUnderSamplingUltimo.csv', index=False)